In [1]:
import os
import cv2
import numpy as np
import pandas as pd
import src.real_bug_dataset
from tqdm.notebook import tqdm
from src.models.pose_resnet import BasicBlock, PoseResNet, Bottleneck
import src.bug_dataset
from src.real_bug_dataset import RealBugDataset
from src.eval.loss import JointsMSELoss
from src.eval.accuracies import accuracy
import torch
from torchvision import transforms
from torch.utils.data import DataLoader
%matplotlib widget
import matplotlib.pyplot as plt
import re
from skimage import io

In [2]:
target_dir = "../data/real_sungaya/"
df = pd.read_csv(os.path.join(target_dir, "realdata.csv"))

In [3]:
# Remove all the datapoints that doesnt have center keypoint visible. 
new_df = pd.DataFrame()
for col, x in df.iterrows():
    vis = x['visibility'].strip('][').split(', ')
    x['visibility'] = list(map(int,vis))
    temp = re.findall(r'\d+', x['key_points_2D'])
    res = list(map(int, temp))
    if len(res) == 56:
        x['key_points_2D'] = np.array(res).reshape((28,2))
        temp = x['key_points_2D'][:,0].copy()
        x['key_points_2D'][:,0] = x['key_points_2D'][:,1]
        x['key_points_2D'][:,1] = temp
        new_df = new_df.append(x)
    
new_df.reset_index(drop=True, inplace=True)    
new_df['visibility'] = new_df['visibility'].apply(np.array)
# Original DF being used

In [9]:
num = 5
sample =new_df.iloc[num]
name = sample['file_name']

img_name = os.path.join(target_dir,
                               sample['file_name'])
        # Reads Image
image = io.imread(img_name)



name =  sample['file_name']
keypoints = sample['key_points_2D']

# keypoints = (sample['key_points_2D']*sungaya_dataset.std_2d)+sungaya_dataset.means_2d
keypoints = sample['key_points_2D']

fig = plt.figure(figsize=(8,6))
ax = fig.add_subplot()

ax.imshow(image.astype('uint8'))
ax.scatter(keypoints[:,0], keypoints[:,1], marker='o', s=5,  color='b')

# ax.set_xlim([0,256])
# ax.set_ylim([256,0])
ax.set_aspect('equal')

plt.title(name)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …